In [1]:
# Extract from https://github.com/StanfordVL/behavior/blob/main/behavior/benchmark/behavior_benchmark.py
import os
import bddl
import json
import behavior
import pyquaternion  
import numpy as np
from PIL import Image
from igibson.utils.utils import parse_config
from igibson.envs.igibson_env import iGibsonEnv
from igibson import object_states

from utils import get_env_config, render_robot_pov, render_360


 _   _____  _  _
(_) / ____|(_)| |
 _ | |  __  _ | |__   ___   ___   _ __
| || | |_ || || '_ \ / __| / _ \ | '_ \
| || |__| || || |_) |\__ \| (_) || | | |
|_| \_____||_||_.__/ |___/ \___/ |_| |_|



In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
PHYSICS_STEPS = 30

scene_id =  "Benevolence_1_int"
task = "cleaning_out_drawers"
env_config = get_env_config()
env_config["scene_id"] = scene_id
env_config["task"] = task
env_config["task_id"] = 0
env_config["instance_id"] = 0

# Keep this in memory as it's removed from the config when we init iGibson - so we need to reload it
robot_name = env_config["robot"]["name"]

env = iGibsonEnv(
        config_file=env_config,
        mode="headless",
        action_timestep=1.0 / 30.0,
        physics_timestep=1.0 / 120.0,
    )

render_robot_pov(env, env_config, step='initial', show=True, save=False)

# Run some steps to let physics settle.
s = env.simulator
for _ in range(PHYSICS_STEPS):
    s.step()
    
render_robot_pov(env, env_config, step='initial', show=True, save=True)
render_360(env, show=True, save=True)

Rendering settings is None - going with the default settings!
																																																																						b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
left_hand_shoulderb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
right_hand_shoulderb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
neckb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data

/home/nicola/miniconda3/envs/igibson/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Angle: 0 - View direction: [ 9.99998383e-01 -3.27519636e-05 -1.79802528e-03]
Image saved at ./images/rendering_attempts/img_0_deg.jpg
Angle: 45 - View direction: [ 0.70712664  0.7070826  -0.00247246]
Image saved at ./images/rendering_attempts/img_45_deg.jpg
Angle: 90 - View direction: [ 2.96979133e-05  9.99998557e-01 -1.69855847e-03]
Image saved at ./images/rendering_attempts/img_90_deg.jpg
Angle: 135 - View direction: [-7.07084638e-01  7.07128920e-01  7.03336537e-05]
Image saved at ./images/rendering_attempts/img_135_deg.jpg
Angle: 180 - View direction: [-9.99998383e-01  3.27519636e-05  1.79802528e-03]
Image saved at ./images/rendering_attempts/img_180_deg.jpg
Angle: 225 - View direction: [-0.70712664 -0.7070826   0.00247246]
Image saved at ./images/rendering_attempts/img_225_deg.jpg
Angle: 270 - View direction: [-2.96979133e-05 -9.99998557e-01  1.69855847e-03]
Image saved at ./images/rendering_attempts/img_270_deg.jpg
Angle: 315 - View direction: [ 7.07084638e-01 -7.07128920e-01 -7.0

# Fix navigate to obj

In [7]:
# Go to cabinet 1
from igibson.action_primitives.starter_semantic_action_primitives import StarterSemanticActionPrimitives

scene = env.scene
s = env.simulator
robot = env.robots[0]
controller = StarterSemanticActionPrimitives(None, scene, robot)
obj = env.task.object_scope['sink.n.01_1']

# Show before
render_robot_pov(env, env_config, step='before', show=True, save=True)

controller.teleport_near_obj(obj)

# Run some steps to let physics settle.
s = env.simulator
for _ in range(PHYSICS_STEPS):
    s.step()
    
# Show after -> didn't work!\
render_robot_pov(env, env_config, step='after', show=True, save=True)

# Old code containing navitagion to object
#for i, action in enumerate(ctrl_gen):
#    robot.apply_action(action)
#    s.step()
#    if i+1 % 100 == 0:
#        print(f"Executing action {action} at step {i}")
#        render_robot_pov(env, env_config, i)

The StarterSemanticActionPrimitive is a work-in-progress and is only provided as an example. It currently only works with BehaviorRobot with its JointControllers set to absolute mode. See provided behavior_robot_mp_behavior_task.yaml config file for an example. See examples/action_primitives for runnable examples.


                                                    
Attempt: 0
                                                        Start collision test.
                                                        Body has collision with objects %s [28]
                                                        Left hand has collision with objects %s [28]
                                                        End collision test.
                                                      Candidate position failed collision test.
                                                    
Attempt: 1
                                                        Start collision test.
                                                        End collision test.


In [8]:
object_names_list = list(env.task.object_scope.keys())
print("object_names_list: ", object_names_list)

object_names_list:  ['bowl.n.01_1', 'bowl.n.01_2', 'cabinet.n.01_1', 'cabinet.n.01_2', 'spoon.n.01_1', 'spoon.n.01_2', 'piece_of_cloth.n.01_1', 'sink.n.01_1', 'floor.n.01_1', 'agent.n.01_1']


In [10]:
def print_properties(obj):
    """
    Requires from igibson import object_states
    """
    logic_state = {}
    #logic_state['InFOVOfRobot'] = obj.states[object_states.InFOVOfRobot].get_value()
    #logic_state['InSameRoomAsRobot'] = obj.states[object_states.InSameRoomAsRobot].get_value()
    #logic_state['InReachOfRobot'] = obj.states[object_states.InReachOfRobot].get_value()

    state_keys = ['InFOVOfRobot', 'InSameRoomAsRobot', 'InReachOfRobot']

    logic_state = {key: obj.states[getattr(object_states, key)].get_value() for key in state_keys}

    print(logic_state)

In [13]:
object_names_list = list(env.task.object_scope.keys())
print("object_names_list: ", object_names_list)

filtered_object_names_list = [name for name in object_names_list \
                              if name.split('.')[0] not in ['agent', 'floor']]
print("filtered_object_names_list: ", filtered_object_names_list)

for object_name in filtered_object_names_list:
    print(f"\nProperties of {object_name}")
    obj = env.task.object_scope[object_name]
    print_properties(obj)
    try:
        ctrl_gen = controller._navigate_to_obj(obj)
        for i, action in enumerate(ctrl_gen):
            print(f"Executing action {action} at step {i}")
            robot.apply_action(action)
            s.step()+
            if i % 5 == 0:
                render_robot_pov(env, env_config, i)
    except Exception as e:
        print(e)
        continue

object_names_list:  ['bowl.n.01_1', 'bowl.n.01_2', 'cabinet.n.01_1', 'cabinet.n.01_2', 'spoon.n.01_1', 'spoon.n.01_2', 'piece_of_cloth.n.01_1', 'sink.n.01_1', 'floor.n.01_1', 'agent.n.01_1']
filtered_object_names_list:  ['bowl.n.01_1', 'bowl.n.01_2', 'cabinet.n.01_1', 'cabinet.n.01_2', 'spoon.n.01_1', 'spoon.n.01_2', 'piece_of_cloth.n.01_1', 'sink.n.01_1']

Properties of bowl.n.01_1
{'InFOVOfRobot': False, 'InSameRoomAsRobot': False, 'InReachOfRobot': True}
Executing action [ 0.00000000e+00 -6.61744490e-23  2.98023224e-07 -1.65814218e-09
 -5.41522804e-18  1.02424461e-26  2.00000003e-01  5.96046448e-08
  9.99999940e-02  0.00000000e+00 -0.00000000e+00  0.00000000e+00
  2.50000000e-01 -2.99999788e-02 -2.50000000e-01  2.92568992e+00
  2.11039401e-01  1.54784117e+00  0.00000000e+00  0.00000000e+00
  2.49999970e-01  2.99999788e-02 -2.50000000e-01 -2.92568992e+00
  2.11039401e-01 -1.54784117e+00  0.00000000e+00  0.00000000e+00] at step 0
Executing action [ 0.00000000e+00 -6.61744490e-23  2.98

In [22]:
bowl1 = env.task.object_scope['bowl.n.01_1']
cabinet1 = env.task.object_scope['cabinet.n.01_1']
bowl1.states[object_states.inside.Inside].get_value(cabinet1)

True